# Galaxy Visualization Metrics Analysis

This notebook provides comprehensive analysis tools for evaluating the performance and quality of the galaxy visualization system. It includes metrics for rendering performance, data processing efficiency, and visualization quality assessment.

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from pathlib import Path
import time
from collections import defaultdict
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set up plotting style
plt.style.use('dark_background')
sns.set_palette("husl")

print("Libraries imported successfully")

## Data Loading and Configuration

Load benchmark data and configure analysis parameters.

In [ ]:
# Configuration parameters
config = {
    'data_dir': '../data',
    'benchmark_results_dir': './benchmark_results',
    'figures_dir': '../docs/figures',
    'target_fps': 60,
    'acceptable_fps': 30,
    'lod_levels': [0, 1, 2]
}

# Create directories if they don't exist
for dir_path in config.values():
    if isinstance(dir_path, str) and 'dir' in dir_path.lower():
        os.makedirs(dir_path, exist_ok=True)

# Load sample benchmark data (simulated for demonstration)
def generate_sample_data():
    """Generate sample benchmark data for analysis"""
    np.random.seed(42)
    
    # Simulate FPS data for different LOD levels
    lod_data = {}
    for lod in config['lod_levels']:
        base_fps = 80 - (lod * 15)  # Higher LOD = lower FPS
        fps_data = np.random.normal(base_fps, 5, 1000)
        fps_data = np.clip(fps_data, 10, 120)  # Reasonable FPS range
        
        lod_data[f'lod_{lod}'] = {
            'fps': fps_data,
            'frame_time': 1000 / fps_data,  # Convert to milliseconds
            'point_count': np.random.randint(10000, 100000 - (lod * 30000), 1000),
            'memory_usage': np.random.normal(256 - (lod * 50), 20, 1000)
        }
    
    return lod_data

sample_data = generate_sample_data()
print("Sample benchmark data generated")

## Performance Analysis

Analyze rendering performance metrics across different LOD levels and scenarios.

In [ ]:
# Performance analysis functions
def analyze_fps_performance(data):
    """Analyze FPS performance across LOD levels"""
    results = {}
    
    for lod_key, lod_data in data.items():
        fps_values = lod_data['fps']
        
        results[lod_key] = {
            'mean_fps': np.mean(fps_values),
            'median_fps': np.median(fps_values),
            'std_fps': np.std(fps_values),
            'min_fps': np.min(fps_values),
            'max_fps': np.max(fps_values),
            'p95_fps': np.percentile(fps_values, 95),
            'p99_fps': np.percentile(fps_values, 99),
            'target_fps_percentage': np.sum(fps_values >= config['target_fps']) / len(fps_values) * 100,
            'acceptable_fps_percentage': np.sum(fps_values >= config['acceptable_fps']) / len(fps_values) * 100
        }
    
    return results

def create_performance_summary(results):
    """Create a summary DataFrame of performance results"""
    df = pd.DataFrame(results).T
    df.index.name = 'LOD Level'
    return df.round(2)

# Analyze performance
performance_results = analyze_fps_performance(sample_data)
performance_summary = create_performance_summary(performance_results)

print("Performance Analysis Summary:")
print(performance_summary)

## Interactive Visualizations

Create interactive plots for exploring performance metrics and visualization quality.

In [ ]:
# Create interactive performance plots
def create_fps_comparison_plot(data):
    """Create an interactive FPS comparison plot"""
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('FPS Distribution', 'Frame Time Distribution', 
                       'FPS vs Point Count', 'Memory Usage'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    
    for i, (lod_key, lod_data) in enumerate(data.items()):
        color = colors[i % len(colors)]
        lod_label = f'LOD {lod_key.split("_")[1]}'
        
        # FPS histogram
        fig.add_trace(
            go.Histogram(x=lod_data['fps'], name=f'{lod_label} FPS', 
                        marker_color=color, opacity=0.7),
            row=1, col=1
        )
        
        # Frame time histogram
        fig.add_trace(
            go.Histogram(x=lod_data['frame_time'], name=f'{lod_label} Frame Time',
                        marker_color=color, opacity=0.7, showlegend=False),
            row=1, col=2
        )
        
        # FPS vs Point Count scatter
        fig.add_trace(
            go.Scatter(x=lod_data['point_count'], y=lod_data['fps'],
                      mode='markers', name=f'{lod_label} Performance',
                      marker=dict(color=color, size=3, opacity=0.6),
                      showlegend=False),
            row=2, col=1
        )
        
        # Memory usage over time
        fig.add_trace(
            go.Scatter(x=list(range(len(lod_data['memory_usage']))), 
                      y=lod_data['memory_usage'],
                      mode='lines', name=f'{lod_label} Memory',
                      line=dict(color=color), showlegend=False),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        title_text="Galaxy Visualization Performance Metrics",
        showlegend=True,
        height=800,
        template="plotly_dark"
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="FPS", row=1, col=1)
    fig.update_xaxes(title_text="Frame Time (ms)", row=1, col=2)
    fig.update_xaxes(title_text="Point Count", row=2, col=1)
    fig.update_xaxes(title_text="Time", row=2, col=2)
    
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Count", row=1, col=2)
    fig.update_yaxes(title_text="FPS", row=2, col=1)
    fig.update_yaxes(title_text="Memory (MB)", row=2, col=2)
    
    return fig

# Create and display the interactive plot
fps_plot = create_fps_comparison_plot(sample_data)
fps_plot.show()

In [ ]:
# Configuration parameters
config = {
    'data_dir': '../data',
    'benchmark_results_dir': './benchmark_results',
    'figures_dir': '../docs/figures',
    'target_fps': 60,
    'acceptable_fps': 30,
    'lod_levels': [0, 1, 2]
}

# Create directories if they don't exist
for dir_path in config.values():
    if isinstance(dir_path, str) and 'dir' in dir_path.lower():
        os.makedirs(dir_path, exist_ok=True)

# Load sample benchmark data (simulated for demonstration)
def generate_sample_data():
    """Generate sample benchmark data for analysis"""
    np.random.seed(42)
    
    # Simulate FPS data for different LOD levels
    lod_data = {}
    for lod in config['lod_levels']:
        base_fps = 80 - (lod * 15)  # Higher LOD = lower FPS
        fps_data = np.random.normal(base_fps, 5, 1000)
        fps_data = np.clip(fps_data, 10, 120)  # Reasonable FPS range
        
        lod_data[f'lod_{lod}'] = {
            'fps': fps_data,
            'frame_time': 1000 / fps_data,  # Convert to milliseconds
            'point_count': np.random.randint(10000, 100000 - (lod * 30000), 1000),
            'memory_usage': np.random.normal(256 - (lod * 50), 20, 1000)
        }
    
    return lod_data

sample_data = generate_sample_data()
print("Sample benchmark data generated")

## Performance Analysis

Analyze rendering performance across different LOD levels and scenarios.

In [ ]:
# Performance analysis functions
class PerformanceAnalyzer:
    def __init__(self, data):
        self.data = data
        self.results = {}
    
    def analyze_fps_distribution(self):
        """Analyze FPS distribution across LOD levels"""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        fig.suptitle('FPS Distribution by LOD Level', fontsize=16, color='white')
        
        for i, lod in enumerate(config['lod_levels']):
            lod_key = f'lod_{lod}'
            fps_data = self.data[lod_key]['fps']
            
            axes[i].hist(fps_data, bins=30, alpha=0.7, color=plt.cm.viridis(i/3))
            axes[i].set_title(f'LOD {lod}', color='white')
            axes[i].set_xlabel('FPS', color='white')
            axes[i].set_ylabel('Frequency', color='white')
            axes[i].axvline(config['target_fps'], color='red', linestyle='--', label='Target FPS')
            axes[i].axvline(config['acceptable_fps'], color='orange', linestyle='--', label='Min Acceptable')
            axes[i].legend()
            axes[i].tick_params(colors='white')
        
        plt.tight_layout()
        plt.show()
        
        # Calculate statistics
        for lod in config['lod_levels']:
            lod_key = f'lod_{lod}'
            fps_data = self.data[lod_key]['fps']
            
            self.results[f'{lod_key}_fps_stats'] = {
                'mean': np.mean(fps_data),
                'median': np.median(fps_data),
                'std': np.std(fps_data),
                'p95': np.percentile(fps_data, 95),
                'p99': np.percentile(fps_data, 99),
                'target_compliance': np.mean(fps_data >= config['target_fps']) * 100,
                'acceptable_compliance': np.mean(fps_data >= config['acceptable_fps']) * 100
            }
    
    def analyze_performance_tradeoffs(self):
        """Analyze performance vs quality tradeoffs"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=['FPS vs Point Count', 'Memory Usage vs LOD', 'Frame Time Distribution', 'Performance Summary'],
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"type": "table"}]]
        )
        
        colors = ['red', 'green', 'blue']
        
        for i, lod in enumerate(config['lod_levels']):
            lod_key = f'lod_{lod}'
            data = self.data[lod_key]
            
            # FPS vs Point Count scatter
            fig.add_trace(
                go.Scatter(
                    x=data['point_count'][:100],  # Sample for visibility
                    y=data['fps'][:100],
                    mode='markers',
                    name=f'LOD {lod}',
                    marker=dict(color=colors[i], alpha=0.6)
                ),
                row=1, col=1
            )
            
            # Memory usage box plot
            fig.add_trace(
                go.Box(
                    y=data['memory_usage'],
                    name=f'LOD {lod}',
                    marker_color=colors[i]
                ),
                row=1, col=2
            )
            
            # Frame time histogram
            fig.add_trace(
                go.Histogram(
                    x=data['frame_time'],
                    name=f'LOD {lod}',
                    opacity=0.7,
                    marker_color=colors[i]
                ),
                row=2, col=1
            )
        
        # Summary table
        summary_data = []
        for lod in config['lod_levels']:
            lod_key = f'lod_{lod}'
            stats = self.results.get(f'{lod_key}_fps_stats', {})
            summary_data.append([
                f'LOD {lod}',
                f"{stats.get('mean', 0):.1f}",
                f"{stats.get('p95', 0):.1f}",
                f"{stats.get('target_compliance', 0):.1f}%"
            ])
        
        fig.add_trace(
            go.Table(
                header=dict(values=['LOD Level', 'Avg FPS', 'P95 FPS', 'Target Compliance']),
                cells=dict(values=list(zip(*summary_data)))
            ),
            row=2, col=2
        )
        
        fig.update_layout(height=800, showlegend=True, title_text="Performance Analysis Dashboard")
        fig.show()

# Create analyzer and run analysis
analyzer = PerformanceAnalyzer(sample_data)
analyzer.analyze_fps_distribution()

In [ ]:
# Run performance tradeoffs analysis
analyzer.analyze_performance_tradeoffs()

## Visualization Quality Metrics

Evaluate the quality and accuracy of the galaxy visualization.

In [ ]:
# Visualization quality assessment
class QualityAnalyzer:
    def __init__(self):
        self.metrics = {}
    
    def simulate_quality_metrics(self):
        """Simulate visualization quality metrics"""
        # Simulate different quality aspects
        quality_data = {
            'spatial_accuracy': np.random.normal(0.95, 0.02, 100),  # How accurately positions are represented
            'color_consistency': np.random.normal(0.92, 0.03, 100),  # Color mapping consistency
            'lod_transition_smoothness': np.random.normal(0.88, 0.05, 100),  # Smoothness of LOD transitions
            'occlusion_handling': np.random.normal(0.85, 0.04, 100),  # How well occlusion is handled
            'visual_completeness': np.random.normal(0.91, 0.03, 100)  # Completeness of visualization
        }
        
        return quality_data
    
    def plot_quality_radar(self, quality_data):
        """Create radar chart for quality metrics"""
        metrics = list(quality_data.keys())
        values = [np.mean(quality_data[metric]) * 100 for metric in metrics]
        
        # Create radar chart
        fig = go.Figure()
        
        fig.add_trace(go.Scatterpolar(
            r=values + [values[0]],  # Close the polygon
            theta=metrics + [metrics[0]],
            fill='toself',
            name='Quality Scores',
            line_color='cyan'
        ))
        
        fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 100]
                )),
            title="Visualization Quality Metrics",
            showlegend=True
        )
        
        fig.show()
    
    def analyze_lod_quality_impact(self):
        """Analyze how LOD affects visualization quality"""
        lod_quality_impact = {}
        
        for lod in config['lod_levels']:
            # Simulate quality degradation with higher LOD
            base_quality = 0.95 - (lod * 0.05)
            quality_scores = np.random.normal(base_quality, 0.02, 50)
            lod_quality_impact[f'LOD_{lod}'] = quality_scores
        
        # Create comparison plot
        fig, ax = plt.subplots(figsize=(10, 6))
        
        positions = range(len(config['lod_levels']))
        quality_data = [lod_quality_impact[f'LOD_{lod}'] for lod in config['lod_levels']]
        
        bp = ax.boxplot(quality_data, positions=positions, patch_artist=True)
        
        colors = ['red', 'yellow', 'green']
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        
        ax.set_xticklabels([f'LOD {lod}' for lod in config['lod_levels']])
        ax.set_ylabel('Quality Score', color='white')
        ax.set_title('Visualization Quality by LOD Level', color='white')
        ax.tick_params(colors='white')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        return lod_quality_impact

# Run quality analysis
quality_analyzer = QualityAnalyzer()
quality_data = quality_analyzer.simulate_quality_metrics()
quality_analyzer.plot_quality_radar(quality_data)
lod_quality = quality_analyzer.analyze_lod_quality_impact()

## Interactive Performance Dashboard

Create interactive visualizations for real-time performance monitoring.

In [ ]:
# Interactive dashboard for real-time monitoring
from IPython.widgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def create_interactive_dashboard():
    """Create interactive performance dashboard"""
    
    @interact(
        lod_level=widgets.IntSlider(min=0, max=2, step=1, value=1, description='LOD Level:'),
        time_window=widgets.IntSlider(min=10, max=100, step=10, value=50, description='Time Window:')
    )
    def update_dashboard(lod_level, time_window):
        lod_key = f'lod_{lod_level}'
        data = sample_data[lod_key]
        
        # Create subplots
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'Real-time Performance Dashboard - LOD {lod_level}', fontsize=16, color='white')
        
        # FPS over time
        fps_sample = data['fps'][:time_window]
        ax1.plot(fps_sample, color='cyan', linewidth=2)
        ax1.axhline(y=config['target_fps'], color='red', linestyle='--', label='Target')
        ax1.axhline(y=config['acceptable_fps'], color='orange', linestyle='--', label='Min Acceptable')
        ax1.set_title('FPS Over Time', color='white')
        ax1.set_ylabel('FPS', color='white')
        ax1.legend()
        ax1.tick_params(colors='white')
        ax1.grid(True, alpha=0.3)
        
        # Frame time histogram
        frame_times = data['frame_time'][:time_window]
        ax2.hist(frame_times, bins=20, alpha=0.7, color='green', edgecolor='white')
        ax2.set_title('Frame Time Distribution', color='white')
        ax2.set_xlabel('Frame Time (ms)', color='white')
        ax2.set_ylabel('Frequency', color='white')
        ax2.tick_params(colors='white')
        
        # Memory usage
        memory_sample = data['memory_usage'][:time_window]
        ax3.plot(memory_sample, color='magenta', linewidth=2)
        ax3.set_title('Memory Usage', color='white')
        ax3.set_ylabel('Memory (MB)', color='white')
        ax3.tick_params(colors='white')
        ax3.grid(True, alpha=0.3)
        
        # Point count vs FPS scatter
        points_sample = data['point_count'][:time_window]
        ax4.scatter(points_sample, fps_sample, alpha=0.6, color='yellow')
        ax4.set_title('Points vs FPS', color='white')
        ax4.set_xlabel('Point Count', color='white')
        ax4.set_ylabel('FPS', color='white')
        ax4.tick_params(colors='white')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Display key statistics
        print(f"\\n=== Performance Summary for LOD {lod_level} ===")
        print(f"Average FPS: {np.mean(fps_sample):.2f}")
        print(f"Min FPS: {np.min(fps_sample):.2f}")
        print(f"Max FPS: {np.max(fps_sample):.2f}")
        print(f"Average Frame Time: {np.mean(frame_times):.2f}ms")
        print(f"Average Points: {np.mean(points_sample):,.0f}")
        print(f"Average Memory: {np.mean(memory_sample):.1f}MB")
        print(f"Target FPS Compliance: {np.mean(fps_sample >= config['target_fps']) * 100:.1f}%")

# Create the dashboard
create_interactive_dashboard()